## team members
### 1- mahmoud abdelmohsen abdelhai
### 2- mostafa mohamed essawi

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge ,Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
%matplotlib inline

### loading data

In [ ]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

### it seems that test data doesn't have casual and registered columns so we have to drop them from the training set

In [ ]:
train.drop(['casual', 'registered'], axis=1, inplace=True)

## EDA

In [ ]:
train.info()

### no null values exit, all features are numeric except datetime which we should change its type to datetime

In [ ]:
train.describe()

In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'], errors='coerce')
test['datetime'] = pd.to_datetime(test['datetime'], errors='coerce')


In [ ]:
train.info()

### extract categorical and numerical variables

In [ ]:
train.columns

In [ ]:
categorical_cols=['season','holiday','workingday','weather']
numerical_cols=['temp','atemp','humidity','windspeed']
label='count'

### create new features (month, year, hour, weekday)

In [ ]:
train['Month'] = train['datetime'].dt.month
test['Month'] = test['datetime'].dt.month
train['Year'] = train['datetime'].dt.year
test['Year'] = test['datetime'].dt.year
train['WeekDay'] = train['datetime'].dt.weekday
test['WeekDay'] = test['datetime'].dt.weekday
train['Hour'] = train['datetime'].dt.hour
test['Hour'] = test['datetime'].dt.hour

In [ ]:
train.head()

In [ ]:
categorical_cols.extend(['Month', 'Year', 'WeekDay', 'Hour'])

### Encoding time data ordered by mean of label in each category

In [ ]:
def encodetime(train,test,col,label):
    d3=train[[col,label]].groupby(col).mean()
    d3.sort_values(by='count',ascending=False)
    plt.scatter(x=d3.index,y=d3['count'])
    plt.xlabel(col)
    d3=d3.sort_values(by='count')
    d3['w']=np.arange(train[col].nunique())
    dic=dict(zip(d3.index,d3['w']))
    train[col]=train[col].map(dic)
    test[col]=test[col].map(dic)

In [ ]:
encodetime(train,test,'Year',label)

In [ ]:
encodetime(train,test,'Month',label)


In [ ]:
encodetime(train,test,'Hour',label)


In [ ]:
encodetime(train,test,'WeekDay',label)


### visualizing Catergorical columns

In [ ]:
def boxplot(x,y,**kwargs):
    sns.boxplot(x=x,y=y)
    x=plt.xticks(rotation=90)
f=pd.melt(train,id_vars=['count'],value_vars=categorical_cols)
g=sns.FacetGrid(f,col='variable',col_wrap=2,sharex=False)
g.map(boxplot,'value','count')

### visualizing numeric columns

In [ ]:
sns.pairplot(train[[*numerical_cols,'count']])

### heatmap

In [ ]:

f, ax = plt.subplots(figsize=(10, 10))
corr = train[[*numerical_cols,'count']].corr()
sns.heatmap(corr,cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax, annot = True)

In [ ]:
f, ax = plt.subplots(figsize=(15, 15))
corr = train.corr()
sns.heatmap(corr,cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax, annot = True)

### temp and atemp columns are highly corrolated
### hour column is highly corrolated with the label(count)
### temp and atemp have 0.39 of corrolation with the label

### Encoding Weather and Season (One Hot Encoding)

In [ ]:
# train=pd.get_dummies(train,columns=['season'])
# test=pd.get_dummies(test,columns=['season'])


In [ ]:
train.columns

In [ ]:
# defining Lag features

features=['holiday', 'workingday', 'weather', 'temp', 'atemp', 'Hour']

def lag(data,features, shift):
    for feature in features:
        data['lag_'+str(shift)+'_'+feature] = data[feature].shift(shift)

In [ ]:
# lag features for train data
for i in range(1,5):
    lag(train, features, i)
    lag(train, features, -i)
    
    
    
# lag features for test data
for i in range(1,5):
    lag(test, features, i)
    lag(test, features, -i)



In [ ]:
# for i in [14,30,60]
#     shifted = temps.shift(1)
#     window = shifted.rolling(window=i)
#     means = window.mean()
#     dataframe = concat([means, temps], axis=1)
#     dataframe.columns = ['mean(t-2,t-1)', 't+1']

In [ ]:
# for i in [14,30,60]:
#     print('Rolling period:', i)
#     train['rolling_mean_'+str(i)] = train.groupby(['id'])['count'].transform(lambda x: x.shift(1).rolling(i).mean())
#     temp_df['rolling_std_'+str(i)]  = temp_df.groupby(['id'])['count'].transform(lambda x: x.shift(1).rolling(i).std())

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.isnull().sum()

In [ ]:
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)

In [ ]:
train.tail()

In [ ]:
train['count'].plot(kind='hist')

### target is skewed left

In [ ]:
## using box cox transform

def trans(x,l1=0.3,l2=0):
    if l1!=0:
        return ((x+l2)**l1-1)/l1
    else:
        return np.log(x+l2)
def rev_trans(x,l1=0.3,l2=0):
    return (x*l1+1)**(1/l1)-l2

z=train[label].apply(trans)   
sns.displot(z , kde=True, height=8.27, aspect=11.7/8.27)

### splitting date into train and validation using time series split

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

X = train.drop(['count','datetime'],axis=1)
xtest = test.drop(['datetime'],axis=1)
y = train['count']

tss = TimeSeriesSplit(n_splits = 5)
for train_index, test_index in tss.split(X):
    X_train, X_valid = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

### splitting date into train and validation using train test split

In [ ]:
xt,xv,yt,yv=train_test_split(X,y,test_size=0.2,random_state=0)

### Model Training and Testing

### Using XGradient Boosting Regressor

In [ ]:
def mk_model_xgb(xt,xv,yt,yv,func1,func2,lr=1,min_child_weight =25,colsample_bytree = 0.8,md=None):
    model =XGBRegressor( colsample_bytree = colsample_bytree, learning_rate = lr,min_child_weight =min_child_weight, max_depth=md )
    ytt=yt.apply(func1)
    model.fit(xt,ytt)
    ypt=np.apply_along_axis(func2,arr=model.predict(xt),axis=0)
    ypv=np.apply_along_axis(func2,arr=model.predict(xv),axis=0)
    print('training r2:',r2_score(yt,ypt))
    print('Validation r2:',r2_score(yv,ypv))
    print('training rmsle:',np.sqrt(msle(yt,ypt)))
    print('validation rmsle:',np.sqrt(msle(yv,ypv)))
    return model

In [ ]:
# applying model on time_series_split
_=mk_model_xgb(X_train,X_valid,y_train,y_valid,func1=trans,func2=rev_trans,lr=0.2,min_child_weight =20,colsample_bytree = 0.8,md=20)

In [ ]:
# applying model on train_test_split
_=mk_model_xgb(xt,xv,yt,yv,func1=trans,func2=rev_trans,lr=0.2,min_child_weight =20,colsample_bytree = 0.8,md=20)

### seems that train test split lead to low rmsle

### train model on the whole dataset

In [ ]:
# fit the model on box cox of the target
model=XGBRegressor(colsample_bytree = 0.8, learning_rate = 0.2,min_child_weight =20, max_depth=20).fit(X,y.apply(trans))


In [ ]:
xtest.head()

In [ ]:
# predict the box cox values of the target
log_y_pred = model.predict(xtest)

In [ ]:
# transform predicted y into its skewed version
yp=np.apply_along_axis(rev_trans,arr=log_y_pred,axis=0)
yp = np.round(yp)

In [ ]:
plt.hist(yp)

In [ ]:
submission = test[['datetime']].copy()
submission['count'] = yp
submission.head()

### generate the csv submission file

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)